In [14]:
import numpy as np

from randgen import gen_autocorrel
import scipy as sp

from edgedroid.models import *
import edgedroid.data as e_data
from gabriel_lego import LEGOTask, FrameResult
from tqdm.notebook import tqdm

model = EdgeDroidModel(
    frame_trace=e_data.load_default_trace("square00", truncate=5),
    frame_model=ZeroWaitSamplingPolicy.from_default_data(),
    timing_model=MultiCurveFittingExecutionTimeModel(neuroticism=1.0)
)

task = LEGOTask(e_data.load_default_task("square00", truncate=5))


def generate_latencies(mode: float, rho: float, min_bound: float = 0.0) -> Iterator[float]:
    prev_value = mode
    while True:
        prev_value = np.random.rayleigh(scale=(mode * (1 - rho)) + (prev_value * rho))
        yield min_bound + prev_value


def emulate_run(model: EdgeDroidModel, task: LEGOTask, rtt_s_iter: Iterator[float], desc: str | None = None):
    step_iter = tqdm(model.play_steps_emulated_time(), total=model.step_count + 1, desc=desc, leave=False)
    
    for model_step in step_iter:
        frame_timings: Optional[FrameTimings] = None
        prev_result: Optional[FrameResult] = None
        while True:
            try:
                model_frame = model_step.send(frame_timings)
            except StopIteration:
                if prev_result != FrameResult.SUCCESS:
                    raise Exception(prev_result)
                break
            prev_result = task.submit_frame(model_frame.frame_data)
            
            rtt = next(rtt_s_iter)
            frame_timings = FrameTimings(rtt * 0.9, rtt * 0.1)
            

emulate_run(model, task, generate_latencies(3.2, rho=.2, min_bound=0.1))
model.model_step_metrics()

  0%|          | 0/6 [00:00<?, ?it/s]

,last_frame_rtt,execution_time,step_duration,time_to_feedback,wait_time,frame_count
step_number,,,,,,
0,4.421520,0.000000,4.421520,4.421520,0.000000,1
1,7.717245,6.543293,16.337220,9.793927,2.076681,4
2,9.450422,6.628337,16.651224,10.022888,0.572465,2
3,6.368832,6.628337,17.435252,10.806916,4.438084,2
4,9.925058,6.628337,17.981392,11.353055,1.427997,3
5,4.989847,6.628337,14.901509,8.273172,3.283325,3


In [23]:
import itertools as it

trace = "square00"
truncate = 50
reps_per_model = 10

timing_models = {
    "curve-high": lambda : MultiCurveFittingExecutionTimeModel(neuroticism=1.0),
    "curve-low": lambda : MultiCurveFittingExecutionTimeModel(neuroticism=0.0),
    "legacy": lambda : LegacyModel(),
    "first-order": lambda : FirstOrderETM(),
    "first-order-median": lambda : FirstOrderAggregateETM(np.median)
}

min_bound = 0.042 # 24FPS
rho_vals = (
    0.0, 
    0.25, 
    0.5
)
sigma_vals = (
    0.1, 
    0.2, 
    0.4, 
    0.8, 
    1.6, 
    3.2
)

dfs = deque()

params_iter = tqdm(list(it.product(
    timing_models.items(),
    range(reps_per_model),
    rho_vals,
    sigma_vals
)), desc="Parameter combinations")
for (model_name, model_constructor), rep, rho, sigma in params_iter:
    model = EdgeDroidModel(
        frame_trace=e_data.load_default_trace(trace, truncate=truncate),
        frame_model=ZeroWaitSamplingPolicy.from_default_data(),
        timing_model=model_constructor()
    )

    task = LEGOTask(e_data.load_default_task(trace, truncate=truncate))
    
    desc = f"{model_name} {rho=} {sigma=} ({rep + 1}/{reps_per_model})"
    emulate_run(model, task, generate_latencies(sigma, rho, min_bound=min_bound), desc=desc)
    
    df = model.model_step_metrics()
    df["model"] = model_name
    df["rep"] = rep
    df["rho"] = rho
    df["sigma"] = sigma
    
    dfs.append(df)
    
data = pd.concat(dfs)
data.to_csv("./per_frame_delay.csv")
data

Parameter combination:   0%|          | 0/900 [00:00<?, ?it/s]

curve-high rho=0.0 sigma=0.1 (1/10):   0%|          | 0/51 [00:00<?, ?it/s]

curve-high rho=0.0 sigma=0.2 (1/10):   0%|          | 0/51 [00:00<?, ?it/s]

curve-high rho=0.0 sigma=0.4 (1/10):   0%|          | 0/51 [00:00<?, ?it/s]

curve-high rho=0.0 sigma=0.8 (1/10):   0%|          | 0/51 [00:00<?, ?it/s]

...

first-order-median rho=0.5 sigma=0.4 (10/10):   0%|          | 0/51 [00:00<?, ?it/s]

first-order-median rho=0.5 sigma=0.8 (10/10):   0%|          | 0/51 [00:00<?, ?it/s]

first-order-median rho=0.5 sigma=1.6 (10/10):   0%|          | 0/51 [00:00<?, ?it/s]

first-order-median rho=0.5 sigma=3.2 (10/10):   0%|          | 0/51 [00:00<?, ?it/s]

,last_frame_rtt,execution_time,step_duration,time_to_feedback,wait_time,frame_count,model,rep,rho,sigma
step_number,,,,,,,,,,
0,0.143041,0.000000,0.143041,0.143041,0.000000,1,curve-high,0,0.0,0.1
1,0.201519,5.004304,5.333957,0.329653,0.128135,33,curve-high,0,0.0,0.1
2,0.143975,5.072477,5.222050,0.149573,0.005599,32,curve-high,0,0.0,0.1
3,0.067107,5.006894,5.083496,0.076602,0.009495,27,curve-high,0,0.0,0.1
4,0.140907,4.585061,4.731451,0.146390,0.005483,33,curve-high,0,0.0,0.1
...,...,...,...,...,...,...,...,...,...,...
46,13.235569,5.230947,19.331939,14.100992,0.865423,2,first-order-median,9,0.5,3.2
47,3.071064,5.230947,8.727258,3.496311,0.425247,4,first-order-median,9,0.5,3.2
48,4.797235,5.230947,10.105197,4.874250,0.077015,3,first-order-median,9,0.5,3.2
